In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

**Read file**

In [4]:
df = pd.read_csv("/content/movies (1).csv")

**Check the information of the dataset**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3803 entries, 0 to 3802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 3803 non-null   int64  
 1   budget                3803 non-null   int64  
 2   genres                3803 non-null   object 
 3   homepage              1421 non-null   object 
 4   id                    3803 non-null   int64  
 5   keywords              3577 non-null   object 
 6   original_language     3802 non-null   object 
 7   original_title        3802 non-null   object 
 8   overview              3801 non-null   object 
 9   popularity            3802 non-null   float64
 10  production_companies  3802 non-null   object 
 11  production_countries  3802 non-null   object 
 12  release_date          3802 non-null   object 
 13  revenue               3802 non-null   float64
 14  runtime               3801 non-null   float64
 15  spoken_languages     

**Check null values**

In [6]:
df.isnull().sum()

,0
index,0
budget,0
genres,0
homepage,2382
id,0
keywords,226
original_language,1
original_title,1
overview,2
popularity,1


**Shape of the dataset**

In [7]:
df.shape

(3803, 24)

**Check top five values**

In [8]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


**Find out the column name**

In [9]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

**Selecting the appropiate columns for training model**

In [10]:
selected_cols=['genres','keywords','tagline','cast','director']
print(selected_cols)

['genres', 'keywords', 'tagline', 'cast', 'director']


**Replacing null values with null string**

In [11]:
for feat in selected_cols:
    df[feat]=df[feat].fillna('')


**Combining all selected feature**

In [12]:
new_df = df['genres'].astype(str) + '' + df['keywords'].astype(str) + '' + df['tagline'].astype(str) + '' + df['cast'].astype(str) + '' + df['director'].astype(str) + '' + df['release_date'].astype(str) + '' + df['runtime'].astype(str) + '' + df['revenue'].astype(str) + '' + df["homepage"].astype(str) + '' + df['tagline'].astype(str) + '' + df['title'].astype(str) + '' + df['vote_average'].astype(str) + '' + df['vote_count'].astype(str)
new_df

,0
0,Action Adventure Fantasy Science Fictioncultur...
1,Adventure Fantasy Actionocean drug abuse exoti...
2,Action Adventure Crimespy based on novel secre...
3,Action Crime Drama Thrillerdc comics crime fig...
4,Action Adventure Science Fictionbased on novel...
...,...
3798,"Comedy DramaAt this wedding, the F-word stands..."
3799,Adventure Action Thriller Crimeterror scotland...
3800,Comedy Fantasy Science Fictionloser bullying l...
3801,Dramaalcohol adultery river camping provinceKo...


**First five values of new data frame**

In [13]:
new_df.head()

,0
0,Action Adventure Fantasy Science Fictioncultur...
1,Adventure Fantasy Actionocean drug abuse exoti...
2,Action Adventure Crimespy based on novel secre...
3,Action Crime Drama Thrillerdc comics crime fig...
4,Action Adventure Science Fictionbased on novel...


**Checking for any null values in the new data frame**

In [14]:
new_df.isnull().sum()

0

In [28]:
df.shape

(3803, 24)

**Converting Text to numerical Value**

In [15]:
#converting text data to numberical values
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
cv_fit = cv.fit_transform(new_df)


In [16]:
print(cv_fit)

  (0, 8709)	1
  (0, 8895)	1
  (0, 18034)	1
  (0, 32539)	1
  (0, 18416)	1
  (0, 13279)	1
  (0, 19389)	1
  (0, 33867)	2
  (0, 37843)	1
  (0, 13542)	1
  (0, 33693)	1
  (0, 35356)	2
  (0, 38846)	2
  (0, 28403)	2
  (0, 28864)	2
  (0, 32191)	1
  (0, 38878)	1
  (0, 39294)	1
  (0, 32152)	1
  (0, 33301)	1
  (0, 38112)	1
  (0, 34288)	1
  (0, 24336)	1
  (0, 26722)	1
  (0, 31408)	1
  :	:
  (3801, 10269)	1
  (3801, 6450)	1
  (3801, 23747)	1
  (3801, 17102)	1
  (3801, 16243)	1
  (3801, 9257)	1
  (3801, 12364)	1
  (3801, 30176)	1
  (3801, 25453)	1
  (3801, 31286)	1
  (3801, 17537)	1
  (3801, 23768)	1
  (3801, 9131)	1
  (3801, 34275)	1
  (3801, 37439)	1
  (3801, 5995)	1
  (3801, 2138)	1
  (3801, 19703)	1
  (3801, 16562)	1
  (3801, 19949)	1
  (3802, 16235)	1
  (3802, 13599)	1
  (3802, 21458)	1
  (3802, 27556)	1
  (3802, 13404)	1


In [20]:
similarity= cosine_similarity(cv_fit)
print(similarity)

[[1.         0.40824829 0.10555008 ... 0.0560112  0.         0.        ]
 [0.40824829 1.         0.09049035 ... 0.01714986 0.         0.        ]
 [0.10555008 0.09049035 1.         ... 0.         0.         0.        ]
 ...
 [0.0560112  0.01714986 0.         ... 1.         0.         0.08944272]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.08944272 0.         1.        ]]


In [27]:
similarity.shape

(3803, 3803)

In [35]:
movie_name=input('enter your favourite movie name: ')

enter your favourite movie name: avatar


**Creating a list of movies given in the dataset**

In [36]:
#creating a list of movies given in the dataset
list_of_all_titles=df['title'].astype(str).to_list()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

**Find close match for movie name given by user**

In [37]:
find_close_match= difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Avatar']


**The closest movie according to search**

In [38]:
close_match=find_close_match[0]
print(close_match)

Avatar


**Find index of movie with title**

In [50]:
index_mov=df[df.title==close_match]["index"].values[0]


**Finding similarity score**

In [52]:
similarity_score=list(enumerate(similarity[index_mov]))

In [53]:
print(similarity_score)

[(0, 1.0), (1, 0.40824829046386296), (2, 0.10555008273018725), (3, 0.13121220505220862), (4, 0.18543130372219696), (5, 0.18190171877724973), (6, 0.07693730481128203), (7, 0.1234930956058247), (8, 0.1688801323682996), (9, 0.15498260496951666), (10, 0.25149772741392845), (11, 0.12071025640383393), (12, 0.1600320096032011), (13, 0.18712029714127992), (14, 0.22407133233117493), (15, 0.10212592127570835), (16, 0.11881770515720091), (17, 0.13894669485837496), (18, 0.1215783580010781), (19, 0.3264173397496434), (20, 0.16516802780306675), (21, 0.0674295649898898), (22, 0.2773796521750473), (23, 0.229865128490995), (24, 0.2692805668394871), (25, 0.039215686274509796), (26, 0.15124753549550501), (27, 0.06063390625908324), (28, 0.1669931442893732), (29, 0.08642716314028857), (30, 0.19105170543026556), (31, 0.24504901470490167), (32, 0.06842857592356093), (33, 0.11669000700233415), (34, 0.0), (35, 0.1921866397915419), (36, 0.15244318679334118), (37, 0.13725490196078427), (38, 0.1252448582170299), 

**Index Numbers of top 10 movies along with their similarity**

In [59]:
print('Index Numbers of top 10 movies along with their similarity')
sorting_similar_mov=sorted(similarity_score,key= lambda x:x[1], reverse=True)
top10_recomended= sorting_similar_mov[:10]
print(top10_recomended)
for i in top10_recomended:
    index=i[0]

Index Numbers of top 10 movies along with their similarity
[(0, 1.0), (1, 0.40824829046386296), (1577, 0.35759926992607577), (1500, 0.3488110957132956), (329, 0.3466209711698763), (156, 0.3457086525611543), (342, 0.34479769273649247), (510, 0.34416981360411797), (206, 0.34064623668312755), (2815, 0.33548786484335924)]
